## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [310]:
#configure directory to import sast libraries
import sys 
import os 
#add sast library path
file_path = os.path.expanduser("~/random_sast/sast")
sys.path.append(file_path)
#add cd_diagram library path
file_path = os.path.expanduser("~/random_sast/cd_diagram")
sys.path.append(file_path)

#sys.path

In [311]:
from sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
import os

### Reading Datasets

In [312]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [313]:
df_other_methods['method'].unique()

array(['SAST', 'ROCKET', 'HIVE-COTEv1', 'BOSS', 'RISE', 'TSF',
       'InceptionTime', 'TS-CHIEF', 'S-BOSS', 'STC', 'ResNet',
       'ProximityForest', 'WEASEL', 'Catch22', 'cBOSS'], dtype=object)

In [314]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsRsast'
#directory = os.getcwd()+'/results_accuracy_per_ds'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        df['rpoint']=df['rpoint'].astype(str)
        df['nb_per_class']=df['nb_per_class'].astype(str)

        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
df_result.head(3)


,Unnamed: 0,accuracy,time,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,dataset_name,classifier_name,rpoint,nb_per_class,method,len_method
0,0,0.573333,0.018448,0.004667,0.003236,0.006212,0.003818,SmoothSubspace,ACF&PACF: n_random_points=1 nb_inst_per_class=1,1,1,Rsast,ACF&PACF
1,1,0.566667,0.012903,0.005363,0.001630,0.001871,0.003664,SmoothSubspace,ACF: n_random_points=1 nb_inst_per_class=1,1,1,Rsast,ACF
2,2,0.613333,0.019624,0.006083,0.003176,0.005806,0.004071,SmoothSubspace,PACF: n_random_points=1 nb_inst_per_class=1,1,1,Rsast,PACF


In [315]:
#df_result=pd.read_csv("df_overall_tunning_SmoothSubspace3.csv")

df_result['dataset_name'].unique()


array(['SmoothSubspace', 'Chinatown', 'SonyAIBORobotSurface2',
       'DistalPhalanxOutlineCorrect', 'GunPoint', 'ItalyPowerDemand',
       'DistalPhalanxOutlineAgeGroup', 'ShapeletSim', 'SyntheticControl',
       'SonyAIBORobotSurface1', 'Coffee', 'BirdChicken'], dtype=object)

In [316]:
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [317]:

filter_param=df_result[df_result.rpoint=='(lenthg ts)//2']
filter_param.rpoint.unique()

array(['(lenthg ts)//2'], dtype=object)

In [318]:

filter_param=filter_param[filter_param.nb_per_class=='(max instances per class)//2']
filter_param.nb_per_class.unique()

array(['(max instances per class)//2'], dtype=object)

In [319]:
# create a pivot table with the mean of score by dataset

len_method_vs_ds=pd.pivot_table(filter_param, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="ROCKET"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('method',axis=1)
merged_df

,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF,dataset,score
0,Chinatown,0.956268,0.973178,0.961322,0.972400,0.961127,0.968902,Chinatown,0.982507
1,Coffee,0.964286,1.000000,0.964286,0.935714,0.971429,1.000000,Coffee,1.000000
2,DistalPhalanxOutlineCorrect,0.664493,0.710145,0.668841,0.691304,0.730435,0.709420,DistalPhalanxOutlineCorrect,0.771739
3,GunPoint,0.825778,0.979556,0.831111,0.967111,0.955111,0.972444,GunPoint,1.000000
4,ItalyPowerDemand,0.945384,0.945578,0.946615,0.945060,0.947716,0.944606,ItalyPowerDemand,0.968902
5,SmoothSubspace,0.825333,0.888444,0.810667,0.874222,0.825778,0.884444,SmoothSubspace,0.980000
6,SonyAIBORobotSurface1,0.752856,0.785469,0.773932,0.705158,0.718469,0.791126,SonyAIBORobotSurface1,0.921797
7,SonyAIBORobotSurface2,0.756278,0.898496,0.760895,0.861630,0.822176,0.898566,SonyAIBORobotSurface2,0.916055


In [320]:
# create a pivot table with the mean of score by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.841472,0.691157,0.758818,0.769146,0.753944,0.767718
ACF&PACF,0.903435,0.769371,0.793374,0.801692,0.794660,0.801711
Max ACF,0.840546,0.680236,0.744893,0.750479,0.727600,0.757632
Max PACF,0.880204,0.699532,0.764194,0.766124,0.748142,0.764230
None,0.869320,0.700765,0.743992,0.745258,0.734713,0.742342
PACF,0.902680,0.772829,0.787058,0.806129,0.804273,0.799287


In [321]:
# create a pivot table with the mean of score by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.839187,0.688101,0.745970,0.773895,0.763327,0.771896
ACF&PACF,0.903470,0.760900,0.782696,0.807996,0.803560,0.805618
Max ACF,0.839069,0.647698,0.743942,0.758819,0.748895,0.763040
Max PACF,0.881344,0.682764,0.766722,0.767201,0.756627,0.767708
None,0.875845,0.666648,0.742882,0.756882,0.740277,0.753513
PACF,0.902937,0.764358,0.784889,0.803765,0.810859,0.805434


#### Time: Subsequence Lenght Method 

In [322]:
# create a pivot table with the mean of the time by dataset
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['dataset_name'], aggfunc='mean')

dataset_name,BirdChicken,Chinatown,Coffee,DistalPhalanxOutlineAgeGroup,DistalPhalanxOutlineCorrect,GunPoint,ItalyPowerDemand,ShapeletSim,SmoothSubspace,SonyAIBORobotSurface1,SonyAIBORobotSurface2,SyntheticControl
len_method,,,,,,,,,,,,
ACF,81.630738,0.120657,43.827708,132.824072,231.141162,42.810175,0.674857,7.645123,1.599955,0.333337,0.815245,128.598058
ACF&PACF,297.280662,0.368780,148.320089,419.191777,679.493913,95.588660,1.649303,222.211859,2.648634,1.902398,3.008572,402.314136
Max ACF,2.745482,0.122675,3.134972,49.805655,76.582509,4.674037,0.681721,2.640422,1.595359,0.318064,0.627533,45.391469
Max PACF,107.885490,0.134552,6.470813,29.798286,46.293955,4.200933,0.747006,124.217464,1.849311,0.345630,0.495278,29.918500
None,1.853694,0.109866,1.756151,28.495936,43.753207,2.811864,0.665972,1.836391,1.571106,0.242769,0.397229,28.232846
PACF,208.928276,0.290148,105.678870,293.491555,452.581422,52.941237,1.651981,215.296044,2.606331,1.603944,2.348842,260.544206


In [323]:
# create a pivot table with the mean of the time by number of instances per class
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,41.374324,2.370840,23.457031,99.368711,49.035581,68.640492
ACF&PACF,119.692646,9.727182,92.841257,319.939252,172.333761,228.019446
Max ACF,12.003714,0.575483,4.900634,30.635086,13.054875,19.903057
Max PACF,7.730573,2.555927,24.915888,39.995921,33.640646,34.146247
None,6.647367,0.406222,3.085724,18.249175,7.967871,11.900655
PACF,78.858843,7.029608,69.667646,221.692364,120.211900,159.979102


In [324]:
# create a pivot table with the mean of the time by number of random points
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,34.934749,1.479961,13.304745,130.142224,39.382845,65.341381
ACF&PACF,98.846568,22.062565,57.138879,407.747755,136.849294,221.005644
Max ACF,8.471901,0.508722,3.931218,37.935515,11.379967,19.031411
Max PACF,5.612837,17.623541,19.241356,42.666004,26.580252,31.372673
None,4.791614,0.324837,2.383123,22.816729,6.839714,11.198669
PACF,64.321160,20.390388,43.460680,279.401390,99.555134,151.075854


#### Variance: Subsequence Lenght Method 

In [325]:
# create a pivot table with the variance of the accuracy by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.009998,0.040966,0.047080,0.040980,0.042145,0.041537
ACF&PACF,0.006736,0.043734,0.049837,0.043452,0.044646,0.045972
Max ACF,0.011198,0.048660,0.046318,0.046647,0.049522,0.043620
Max PACF,0.009564,0.057702,0.047982,0.051605,0.055137,0.052333
None,0.008806,0.053091,0.052258,0.049727,0.055322,0.049404
PACF,0.006327,0.039945,0.049298,0.045674,0.039032,0.044683


In [326]:
# create a pivot table with the variance of the accuracy by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='var')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.010150,0.049396,0.042842,0.038232,0.043131,0.039355
ACF&PACF,0.006774,0.050307,0.045447,0.043315,0.045591,0.043876
Max ACF,0.010705,0.050599,0.046682,0.043265,0.056796,0.042991
Max PACF,0.009893,0.062107,0.048823,0.048286,0.059011,0.048753
None,0.009935,0.061072,0.050655,0.047695,0.054905,0.050382
PACF,0.006620,0.044150,0.049773,0.039370,0.041621,0.044168


#### Another Results

In [327]:
df_result[['dataset_name','len_method','classifier_name']].describe()

,dataset_name,len_method,classifier_name
count,6900,6900,6900
unique,12,6,216
top,SmoothSubspace,ACF&PACF,None: n_random_points= (lenthg ts)//2 nb_inst_...
freq,720,1150,100


In [328]:
pd.pivot_table(df_result, values='time', index=['dataset_name'], aggfunc='sum')/(60*60)

,time
dataset_name,
BirdChicken,14.590090
Chinatown,0.038223
Coffee,3.435429
DistalPhalanxOutlineAgeGroup,19.866818
DistalPhalanxOutlineCorrect,38.246154
GunPoint,6.767564
ItalyPowerDemand,0.202361
ShapeletSim,11.955152
SmoothSubspace,0.395690


In [329]:
df_result['time'].sum()/(60*60)

114.55792629732026

#### Generate CD Diagram Tunning Hyperparameter

In [330]:
#create dataset to generate cd diagram
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
#df_perf.classifier_name.unique()


In [331]:
param_per_ds=pd.pivot_table(df_perf, index=['dataset_name'],values=['classifier_name'] ,aggfunc='count')
param_per_ds=param_per_ds.reset_index()
complete_ds=param_per_ds[param_per_ds.classifier_name==max(param_per_ds.classifier_name)]["dataset_name"].unique()

df_perf=df_perf[df_perf.dataset_name.isin(complete_ds)]
df_result=df_result[df_result.dataset_name.isin(complete_ds)]
df_perf.dataset_name.unique()

array(['Chinatown', 'Coffee', 'DistalPhalanxOutlineCorrect', 'GunPoint',
       'ItalyPowerDemand', 'SmoothSubspace', 'SonyAIBORobotSurface1',
       'SonyAIBORobotSurface2'], dtype=object)

In [332]:
from cd_function import *
#generate hyperparameter tuning cd diagrams
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF&PACF:')], labels=True, title="ACF&PACF comparison", name_file='images_cd_diagram/cd-diagram_ACF&PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^PACF:')], labels=True, title="PACF comparison", name_file='images_cd_diagram/cd-diagram_PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF:')], labels=True, title="ACF comparison", name_file='images_cd_diagram/cd-diagram_ACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^None:')], labels=True, title="None comparison", name_file='images_cd_diagram/cd-diagram_None.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max ACF:')], labels=True, title="Max ACF comparison", name_file='images_cd_diagram/cd-diagram_MaxACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max PACF:')], labels=True, title="Max PACF comparison", name_file='images_cd_diagram/cd-diagram_MaxPACF.png')
draw_cd_diagram(df_perf, labels=True, title="overall Hyperparameter Tunning", name_file='images_cd_diagram/cd-diagram_overall_tunning.png')



/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small 

In [333]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


,0
ACF: n_random_points=1 nb_inst_per_class=1,196.7500
Max ACF: n_random_points=1 nb_inst_per_class=1,195.6875
Max PACF: n_random_points=1 nb_inst_per_class=1,188.8125
ACF: n_random_points=10 nb_inst_per_class=1,186.6250
Max ACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=1,184.5625
...,...
PACF: n_random_points=100 nb_inst_per_class=30,43.8750
ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=50,42.8750
ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=30,42.3750
ACF&PACF: n_random_points=50 nb_inst_per_class=100,41.1875


In [334]:
#generate example cd diagrams
#draw_cd_diagram(pd.read_csv('example_cd_diagram.csv'), labels=True, title="Example CD Diagram", name_file='cd-diagram_example.png')

#### Focus on most accurate lenght method

##### Overall Accuracy

In [335]:
# highest accurate hyperparameters
average_ranks[average_ranks[0]==min_rank][0].index

Index(['ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=100'], dtype='object')

In [336]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.906115,0.851571,0.900476,0.918048,0.915469,0.918209
1,0.887970,0.778297,0.768475,0.835488,0.836532,0.839840
10,0.906002,0.809821,0.838031,0.857332,0.852875,0.852384
100,0.907645,0.813213,0.843813,0.857833,0.857874,0.858737
30,0.900178,0.784446,0.847454,0.856690,0.850254,0.857721
50,0.902329,0.818998,0.841610,0.867605,0.859243,0.862244


In [337]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.006529,0.008913,0.006878,0.005580,0.006038,0.006032
1,0.008781,0.015761,0.052589,0.032691,0.033129,0.032690
10,0.006343,0.026288,0.034497,0.035496,0.034958,0.035030
100,0.006417,0.032199,0.034619,0.035555,0.035657,0.035691
30,0.007660,0.049592,0.035071,0.035176,0.034767,0.035634
50,0.007000,0.032384,0.034306,0.030329,0.035939,0.036193


##### Overall time complexity

In [338]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='mean')/60

,cweights_time,fsubsequence_time,tclassifier_time,tdataset_time,time
dataset_name,,,,,
Chinatown,0.000153,0.001468,0.000136,0.004364,0.006146
Coffee,0.001170,0.102598,0.000719,2.367384,2.472001
DistalPhalanxOutlineCorrect,0.000789,0.029408,0.014176,11.280322,11.324899
GunPoint,0.000702,0.035006,0.001057,1.556273,1.593144
ItalyPowerDemand,0.000133,0.002412,0.000269,0.024644,0.027488
SmoothSubspace,0.000103,0.002886,0.000560,0.040557,0.044144
SonyAIBORobotSurface1,0.000361,0.004451,0.000191,0.026673,0.031707
SonyAIBORobotSurface2,0.000332,0.004047,0.000192,0.045540,0.050143


In [339]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,2.170001,0.060256,0.639046,2.083664,1.309212,1.532242
1,0.073041,0.014015,0.022523,0.111114,0.049283,0.067691
10,0.488461,0.015584,0.148757,0.910384,0.374411,0.535486
100,4.724280,0.144624,1.387989,9.234059,3.490896,5.167018
30,1.422373,0.043514,0.407962,2.756711,1.066451,1.551607
50,2.390613,0.072864,0.660858,4.587690,1.744799,2.569607


##### Calculate weights time complexity

In [340]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.019267,0.020583,0.019752,0.019327,0.018123,0.020405
1,0.022916,0.021810,0.017687,0.026827,0.021387,0.021696
10,0.019024,0.023253,0.020182,0.027211,0.023891,0.023374
100,0.017850,0.029895,0.025756,0.026477,0.028243,0.027430
30,0.019597,0.024953,0.024064,0.027544,0.023103,0.028944
50,0.020675,0.027583,0.025132,0.032290,0.026611,0.032012


##### Finding subsequences time complexity

In [341]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.558255,0.061089,0.524378,0.862964,0.873220,0.815816
1,0.885135,0.195523,0.428400,0.964010,0.782507,0.861997
10,0.625768,0.059261,0.520254,1.157898,0.914289,1.020075
100,0.582417,0.125785,1.182582,3.083040,2.157882,2.482086
30,0.553907,0.075126,0.659957,1.511306,1.232034,1.346904
50,0.523631,0.083268,0.709270,1.966646,1.364101,1.706553


##### Transform Dataset time complexity

In [342]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,129.512743,3.514877,37.762175,124.038984,77.597356,91.020227
1,3.450065,0.601855,0.846299,5.596507,2.101055,3.112014
10,28.618315,0.823863,8.313663,53.323665,21.437139,30.997415
100,282.676870,8.451392,81.944114,550.397529,207.026798,307.193945
30,84.697389,2.456402,23.720532,163.662934,62.608425,91.583688
50,142.777656,4.188686,38.836361,272.974878,103.142713,152.252327


##### Classifier time complexity

In [343]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.102650,0.017599,0.033000,0.088032,0.057789,0.071176
1,0.023354,0.018855,0.058041,0.078222,0.051165,0.064635
10,0.041955,0.027544,0.070463,0.112159,0.087734,0.086692
100,0.173333,0.066589,0.123713,0.526043,0.234946,0.310336
30,0.067391,0.052980,0.071704,0.197493,0.121003,0.132812
50,0.110408,0.070257,0.078900,0.282634,0.151333,0.180211
